In [108]:
import numpy as np
import math
import sympy as sp
from scipy import optimize

In [109]:
#constantes fisicas
g = 9.81 #gravedad
visc_c = 1.14 * 10**(-6) # coeficiente de viscosidad cinematica 15 grados


In [110]:
# datos a colocar (d(mm), L(m), k(mm), K)
d_tuberias = np.array([
    (600, 423, 0.3, 4.2),
    (500, 174, 0.0015, 3.4),
    (300, 373, 0.0015, 5.3),
    (250, 121, 0.0015, 7.5)
])
sp.Matrix(d_tuberias)

Matrix([
[600.0, 423.0,    0.3, 4.2],
[500.0, 174.0, 0.0015, 3.4],
[300.0, 373.0, 0.0015, 5.3],
[250.0, 121.0, 0.0015, 7.5]])

In [111]:
# datos tuberias caudal extremo final (l/s)
q_tuberias = np.array([
    60,
    74,
    60,
])/1000
sp.Matrix(q_tuberias)

Matrix([
[ 0.06],
[0.074],
[ 0.06]])

In [112]:
class Tuberias:
    #constantes físicas
    g = 9.81 #gravedad
    visc_c = 1.14 * 10**(-6) # coeficiente de viscosidad cinematica 15 grados

    def __init__(self,d_tuberias,q_tuberias,Ht):
        self.d_tuberias = d_tuberias
        self.q_tuberias = q_tuberias
        self.Ht = Ht

    def h_f1(self):
        d = 0
        for i in range(len(self.d_tuberias)):
            d += self.d_tuberias[i,1]/((self.d_tuberias[i,0]/1000)**5)
        return self.Ht * (self.d_tuberias[0,1]/(self.d_tuberias[0,0]/1000)**5)/d

    def v_1(self):
        D1_metros = self.d_tuberias[0,0]/1000
        k1_metros = self.d_tuberias[0,2]/1000
        m = math.sqrt(2*g*D1_metros*self.h_f1())
        return ((-2 * m)/math.sqrt(self.d_tuberias[0,1]))*\
                    math.log10((k1_metros/(3.7*D1_metros))+\
                         (2.51*visc_c*math.sqrt(self.d_tuberias[0,1]))/(D1_metros*m))

    def q_n(self):
        q_n = np.empty(len(self.d_tuberias))
        q = 0
        for i in range(len(self.q_tuberias)):
            q += self.q_tuberias[i]
            q_n[i+1] = q_n[0]-q
        q_n[0] = math.pi/4 * (self.d_tuberias[0,0]/1000)**2 * self.v_1()
        return q_n

    def v_n(self):
        v_n = np.empty(len(self.d_tuberias))
        for i in range(len(self.d_tuberias)):
            v = 4 * self.q_n()[i] / (math.pi*(self.d_tuberias[i,0]/1000)**2)
            v_n[i] = v
        v_n[0] = self.v_1()
        return v_n

    def n_reynolds(self):
        re_n = np.empty(len(self.d_tuberias))
        for i in range(len(self.d_tuberias)):
            re = (self.d_tuberias[i,0] * self.v_n()[i]) / (visc_c*1000)
            re_n[i] = re
        return re_n

    def f_n(self):
        f_nc = lambda f,n : 1.0 / np.sqrt(f) + 2.0 * np.log10(
            (self.d_tuberias[n, 2] / self.d_tuberias[n, 0]) / 3.7 + 2.51 / (self.n_reynolds()[n] * np.sqrt(f)))
        f_n = [optimize.brentq(f_nc, 0.0000001, 1, args= (i)) for i in range(len(self.d_tuberias))]
        f_n = np.array(f_n)
        return f_n

    def h_fn(self):
        h_fn = np.empty(len(self.d_tuberias))
        h_fn[0] = self.h_f1()
        for i in range(len(self.d_tuberias)):
            h_f = self.f_n()[i] * self.d_tuberias[i,1] * self.v_n()[i]**2 / ((self.d_tuberias[i,0]/1000) * 2 * g)
            h_fn[i] = h_f
        return h_fn

    def h_mn(self):
        h_mn = np.empty(len(self.d_tuberias))
        for i in range(len(self.d_tuberias)):
            h_m = self.d_tuberias[i,3] * self.v_n()[i]**2 / (2 * g)
            h_mn[i] = h_m
        return h_mn

    def s_hn(self):
        return np.sum(self.h_fn() + self.h_mn())

    def nh_f1(self):
        d = 0
        for i in range(len(self.d_tuberias)):
            d += self.d_tuberias[i,1] / ((self.d_tuberias[i,0]/1000)**5)
        dif_hf = (self.Ht-self.s_hn()) * (self.d_tuberias[0,1]/(self.d_tuberias[0,0]/1000)**5)/d
        nh_f1 = self.h_f1() + dif_hf
        return nh_f1

In [116]:
# definimos los datos de la tuberia y los caudales asi como la altura geometrica
iteracion1 = Tuberias(d_tuberias,q_tuberias, 28.5)
iteracion1.nh_f1()

0.9611758043008491